In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from wdtools import *

In [3]:
setID = 'Set002'; nm_to_add=1420

In [4]:
wd = combine_wd_tables(setID=setID, nm_to_add=nm_to_add)

In [5]:
toCheck_df = wd[wd.wetdet_delin_number.isin(['WD2017-0198', 'WD2017-0229', 'WD2019-0082'])]

In [6]:
review_loop_r1(df=toCheck_df, partial=True, idx=True)

0.0% digitized, 3 records remained, expected to be done in about 2 hours...
WD2017-0229
index = 3
     county     trsqq                   parcel_id   latitude   longitude  \
2     Baker  07S40E00  Many tax maps and tax lots  45.831088 -119.623309   
3     Baker  10S40E00  Many tax maps and tax lots  45.831088 -119.623309   
4     Baker  09S41E00  Many tax maps and tax lots  45.831088 -119.623309   
5     Baker  09S40E00  Many tax maps and tax lots  45.831088 -119.623309   
6     Baker  08S40E00  Many tax maps and tax lots  45.831088 -119.623309   
7     Baker  10S41E00  Many tax maps and tax lots  45.831088 -119.623309   
8     Baker  11S43E00  Many tax maps and tax lots  45.831088 -119.623309   
9     Baker  15S45E00  Many tax maps and tax lots  45.831088 -119.623309   
10    Baker  10S42E00  Many tax maps and tax lots  45.831088 -119.623309   
11    Baker  14S45E00  Many tax maps and tax lots  45.831088 -119.623309   
12    Baker  14S44E00  Many tax maps and tax lots  45.831088 -119.